In [1]:
from importlib.metadata import version

import tiktoken
import torch

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.2.2
tiktoken version: 0.6.0


In [4]:
with open("/workspaces/upper-division-cs/ai-24sp/assignments/whereismyprozac/data/totally-profound-monologue.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 48658
7/27/1978

When I was 18... 18 years old, I saw for the first time in my life... I saw an image of 


In [8]:
import re

text = "The cat has the pipe!"
result = re.split(r'(\s)', text)

print(result)

['The', ' ', 'cat', ' ', 'has', ' ', 'the', ' ', 'pipe!']


In [9]:
result = re.split(r'([,.]|\s)', text)

print(result)

['The', ' ', 'cat', ' ', 'has', ' ', 'the', ' ', 'pipe!']


In [10]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['The', 'cat', 'has', 'the', 'pipe!']


In [11]:
text = "The cat has- the pipe!?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['The', 'cat', 'has-', 'the', 'pipe', '!', '?']


In [12]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['7/27/1978', 'When', 'I', 'was', '18', '.', '.', '.', '18', 'years', 'old', ',', 'I', 'saw', 'for', 'the', 'first', 'time', 'in', 'my', 'life', '.', '.', '.', 'I', 'saw', 'an', 'image', 'of', 'clarity']


In [13]:
print(len(preprocessed))

11852


In [14]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)

print(vocab_size)

2105


In [15]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [16]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
('*Okamoto', 5)
('*fumbles', 6)
('*puts', 7)
(',', 8)
('-', 9)
('-Oscar', 10)
('.', 11)
('000', 12)
('1', 13)
('1-6', 14)
('10', 15)
('1111/j', 16)
('12th', 17)
('1330]', 18)
('1600-0773', 19)
('18', 20)
('1978', 21)
('1980', 22)
('1981', 23)
('1981*', 24)
('1983', 25)
('1994', 26)
('25%', 27)
('27th', 28)
('33', 29)
('6b', 30)
('7/27/1978', 31)
('75', 32)
('7971729', 33)
('7:27:78]', 34)
('80', 35)
(':', 36)
('?', 37)
('A', 38)
('A:', 39)
('ALWAYS', 40)
('AND', 41)
('Add', 42)
('Africa', 43)
('After', 44)
('Again', 45)
('Ages', 46)
('Ah', 47)
('Aha', 48)
('All', 49)
('Although', 50)


In [23]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = "Lasagna".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?"()\'])', r'\1', text)
        return text

In [24]:
tokenizer = SimpleTokenizerV1(vocab)

text = """" Cat"""
ids = tokenizer.encode(text)
print(ids)

[1, 79]


In [25]:
tokenizer.decode(ids)

'"LasagnaCat'

In [26]:
tokenizer.decode(tokenizer.encode(text))

'"LasagnaCat'

In [27]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [28]:
len(vocab.items())

2107

In [29]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('your', 2102)
('yourself', 2103)
('youthful', 2104)
('<|endoftext|>', 2105)
('<|unk|>', 2106)


In [35]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [36]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Jon is depicted as being grounded in the material world..."
text2 = "a world of things; he is surrounded by objects, and he touches these objects, he interacts with them."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Jon is depicted as being grounded in the material world... <|endoftext|> a world of things; he is surrounded by objects, and he touches these objects, he interacts with them.


In [37]:
tokenizer.encode(text)

[157,
 1116,
 693,
 397,
 444,
 961,
 1068,
 1905,
 1243,
 2081,
 11,
 11,
 11,
 2105,
 303,
 2081,
 1367,
 1920,
 983,
 1116,
 1865,
 500,
 1357,
 8,
 364,
 983,
 1960,
 1915,
 1357,
 8,
 983,
 1105,
 2069,
 1907,
 11]

In [38]:
tokenizer.decode(tokenizer.encode(text))


'Jon is depicted as being grounded in the material world... <|endoftext|> a world of things; he is surrounded by objects, and he touches these objects, he interacts with them.'